# PSO ALGORITHM

In [8]:
from softpy.evolutionary.candidate import FloatVectorCandidate
from typing import Callable
from softpy.evolutionary.singlestate import MetaHeuristicsAlgorithm
import numpy as np
import random

In [9]:
class ParticleCandidate(FloatVectorCandidate):
    """
    A candidate for particle swarm optimization (PSO) that represents a particle
    in the swarm with position and velocity attributes.
    """
    
    def __init__(self, size: int, lower: np.ndarray, upper: np.ndarray, candidate: np.ndarray, velocity: np.ndarray, 
                 inertia: float, w_l: float, w_n: float, w_g: float):
        # Call parent constructor
        super().__init__(size, candidate, None, lower=lower[0], upper=upper[0], intermediate=False, mutate=None, recombine=None)
        
        if w_l + w_n + w_g != 1.0:
            raise ValueError("w_l + w_n + w_g must equal 1.0")
        
        # Initialize particle-specific attributes
        self.size = size
        self.lower = lower.astype(float)
        self.upper = upper.astype(float)
        self.candidate = candidate.astype(float)
        self.velocity = velocity.astype(float)
        self.inertia = inertia
        self.w_l = w_l
        self.w_n = w_n
        self.w_g = w_g

    @staticmethod
    def generate(size: int, lower: np.ndarray, upper: np.ndarray, inertia: float, 
                 w_l: float, w_n: float, w_g: float):
        # Generate a random candidate within the bounds  
        candidate = np.random.uniform(lower, upper, size)
        
        # Generate velocity within the specified range
        velocity_range = np.abs(upper - lower)
        velocity = np.random.uniform(-velocity_range, velocity_range, size)
        
        # Create and return particle instance
        return ParticleCandidate(size, lower, upper, candidate, velocity, inertia, w_l, w_n, w_g)
    
    def copy(self):
        """Create a copy of this particle"""
        return ParticleCandidate(
            self.size,
            self.lower.copy(),
            self.upper.copy(),
            self.candidate.copy(),
            self.velocity.copy(),
            self.inertia,
            self.w_l,
            self.w_n,
            self.w_g
        )
    
    def mutate(self):
        # Update candidate position by adding velocity
        self.candidate += self.velocity
    
    def recombine(self, local_best, neighborhood_best, best):
        """Update velocity according to PSO formula"""
        r_l = random.uniform(0, 1)
        r_n = random.uniform(0, 1)
        r_g = random.uniform(0, 1)
        
        self.velocity = (
            self.inertia * self.velocity
            + r_l * self.w_l * (local_best.candidate - self.candidate)
            + r_n * self.w_n * (neighborhood_best.candidate - self.candidate)
            + r_g * self.w_g * (best.candidate - self.candidate)
        )

In [10]:
class ParticleSwarmOptimizer(MetaHeuristicsAlgorithm):
    ''' Represents the process updating particles in a swarm. '''
    
    def __init__(self, fitness_func, pop_size: int, n_neighbors: np.ndarray, 
                 lower: np.ndarray, upper: np.ndarray, inertia: float, 
                 w_l: float, w_n: float, w_g: float, **kwargs):

        super().__init__(candidate_type=None, fitness_func=fitness_func, **kwargs)
        
        self.pop_size = pop_size
        self.fitness_func = fitness_func
        self.n_neighbors = n_neighbors
        self.particles = []
        
        # Store PSO-specific parameters
        self.lower = lower
        self.upper = upper
        self.inertia = inertia
        self.w_l = w_l
        self.w_n = w_n
        self.w_g = w_g
        
        self.best = np.array([None] * pop_size)  # Array of ParticleCandidate instances for each particle's best position
        self.fitness_best = np.full(pop_size, -1000.0, dtype=float)  # Array of best fitness values for each particle
        self.global_best = None  # ParticleCandidate with best position found so far
        self.global_fitness_best = -1000.0  # Best fitness value found so far
    
    def fit(self, n_iters):
        # Create population
        population = []
        for _ in range(self.pop_size):
            particle = ParticleCandidate.generate(
                size=2,
                lower=self.lower,
                upper=self.upper,
                inertia=self.inertia,
                w_l=self.w_l,
                w_n=self.w_n,
                w_g=self.w_g
            )
            population.append(particle)
        
        self.particles = population
        
        for i in range(n_iters):
            # Evaluate fitness and update bests
            for j in range(self.pop_size):
                candidate = population[j]
                fitness = self.fitness_func(candidate)

                # Update local best
                if fitness > self.fitness_best[j]:
                    self.fitness_best[j] = fitness
                    self.best[j] = candidate.copy()

                # Update global best
                if self.global_best is None or fitness > self.global_fitness_best:
                    self.global_best = candidate.copy()
                    self.global_fitness_best = fitness
            
            # Update velocities and positions for each particle
            for j in range(self.pop_size):
                if self.best[j] is None:
                    continue
                    
                # Find neighborhood best
                possible_neighbors = list(range(self.pop_size))
                possible_neighbors.remove(j)
                
                n_neighbors_count = min(self.n_neighbors[j], len(possible_neighbors))
                if n_neighbors_count > 0:
                    neighbor_indices = np.random.choice(possible_neighbors, size=n_neighbors_count, replace=False)
                    
                    best_neighbor_idx = None
                    best_neighbor_fitness = -1000.0
                    
                    for neighbor_idx in neighbor_indices:
                        if self.best[neighbor_idx] is None:
                            continue
                        neighbor_fitness = self.fitness_best[neighbor_idx]
                        if neighbor_fitness > best_neighbor_fitness:
                            best_neighbor_fitness = neighbor_fitness
                            best_neighbor_idx = neighbor_idx
                    
                    if best_neighbor_idx is not None and self.global_best is not None:
                        neighborhood_best = self.best[best_neighbor_idx]
                        population[j].recombine(self.best[j], neighborhood_best, self.global_best)
                        population[j].mutate()
        

# Try to apply this algorithm to a sample population

In [11]:
# Simple and fun PSO tests! 🚀

# Fun test function 1: Find the treasure! 
def treasure_hunt(particle):
    """Find the treasure at position [3, 7]"""
    treasure_x, treasure_y = 3.0, 7.0
    x, y = particle.candidate[0], particle.candidate[1]
    
    # Distance to treasure (closer = higher fitness)
    distance = np.sqrt((x - treasure_x)**2 + (y - treasure_y)**2)
    return -distance  # Negative because closer is better

In [12]:
# Simple setup
pop_size = 10
dimensions = 2  # Just 2D for simplicity

# Search area: 0 to 10 in both directions
lower = np.array([0.0, 0.0])
upper = np.array([10.0, 10.0])

# Each particle has 2-4 neighbors
neighbors = np.array([3, 2, 4, 3, 2, 3, 4, 2, 3, 3])

# Simple PSO weights that sum to 1
w_l, w_n, w_g = 0.3, 0.3, 0.4
inertia = 0.5

print(f"🔍 Searching in area: {lower} to {upper}")
print(f"👥 {pop_size} particles, each with 2-4 neighbors")

🔍 Searching in area: [0. 0.] to [10. 10.]
👥 10 particles, each with 2-4 neighbors


In [13]:
# Test 1: Treasure Hunt! 💰
print("🏴‍☠️ TREASURE HUNT!")
print("Goal: Find treasure at position [3, 7]")

try:
    test_particle = ParticleCandidate.generate(
        size=2, 
        lower=lower, 
        upper=upper, 
        inertia=0.5, 
        w_l=0.3, 
        w_n=0.3, 
        w_g=0.4
    )
    print(f"✅ Created test particle at position: {test_particle.candidate}")
    print(f"🏃 Particle velocity: {test_particle.velocity}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Hint: The generate method needs @staticmethod decorator!")    # Simple test of treasure hunt function
    if 'test_particle' in locals():
        fitness = treasure_hunt(test_particle)
        distance_to_treasure = np.sqrt((test_particle.candidate[0] - 3)**2 + (test_particle.candidate[1] - 7)**2)
        print(f"🗺️  Particle is {distance_to_treasure:.2f} units from treasure")
        print(f"🎯 Fitness score: {fitness:.2f} (higher is better)")

🏴‍☠️ TREASURE HUNT!
Goal: Find treasure at position [3, 7]
✅ Created test particle at position: [9.58495908 6.75952455]
🏃 Particle velocity: [-1.73487458 -4.99003864]


In [14]:
# Run the full PSO treasure hunt! 🏴‍☠️
print("\n🏴‍☠️ FULL TREASURE HUNT WITH PSO!")
print("Let's see if our swarm can find the treasure at [3, 7]!")

try:
    pso_treasure = ParticleSwarmOptimizer(
        fitness_func=treasure_hunt,
        pop_size=pop_size,
        n_neighbors=neighbors,
        lower=lower,
        upper=upper,
        inertia=inertia,
        w_l=w_l,
        w_n=w_n,
        w_g=w_g
    )
    
    print("🚀 Starting treasure hunt optimization...")
    pso_treasure.fit(n_iters=20)  # Run for 20 iterations
    
    if pso_treasure.global_best:
        best_pos = pso_treasure.global_best.candidate
        print(f"💰 TREASURE FOUND!")
        print(f"🎯 Best position: [{best_pos[0]:.2f}, {best_pos[1]:.2f}]")
        print(f"🏆 Best fitness: {pso_treasure.global_fitness_best:.2f}")
        
        # How close did we get?
        distance = np.sqrt((best_pos[0] - 3)**2 + (best_pos[1] - 7)**2)
        print(f"📏 Distance from treasure: {distance:.2f} units")
        
        if distance < 1.0:
            print("🎉 EXCELLENT! Very close to the treasure!")
        elif distance < 2.0:
            print("👍 GOOD! Pretty close to the treasure!")
        else:
            print("🤔 Hmm, still searching...")
    else:
        print("❌ No treasure found yet!")
        
except Exception as e:
    print(f"❌ Error during treasure hunt: {e}")


🏴‍☠️ FULL TREASURE HUNT WITH PSO!
Let's see if our swarm can find the treasure at [3, 7]!
🚀 Starting treasure hunt optimization...
💰 TREASURE FOUND!
🎯 Best position: [3.00, 7.00]
🏆 Best fitness: -0.00
📏 Distance from treasure: 0.00 units
🎉 EXCELLENT! Very close to the treasure!
